##RAG
WHY RAG ?
- we Know llm right?
- but we also know that they are trained on large datasets but not the data set specific to us or any company or not about dataset of today or tomorrow
- here comes concept of RAG 
- Before that why not use Finetuning instead of RAG ?
- yes it is true but finetuning is not always possible as it needs lots of computational power as well as time to train the model
- and also trainning of finetune model regularly for updating data is not possible 
- here comes the concept of RAG 
- RAG uses Vector DB to store data in form of vector embedding and uses retriever to retrieve data from vector DB this data is then used by llm to generate response

## RAG PIPELINE

- Document Loader : Loads all the documents from a directory
- Text Splitter : splits document into smaller chunks for better rag performance
- Embedding Manager : generates embeddings for text
- Vector Store : stores embeddings in vector db
- RAG Retriver : Handles query based retrieval from the vector store
- RAG Pipeline : combines all the above components to create a RAG pipeline


In [92]:
from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from pathlib import Path

In [93]:
def process_all_pdfs(pdf_dir):
    """process all the pdf files in a directory"""
    all_docs = []
    pdf_dir = Path(pdf_dir)

    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} pdf to process")

    for pdf in pdf_files:
        print("processing")
        try:
            loader = PyPDFLoader(str(pdf))
            document = loader.load()

            for doc in document:
                doc.metadata['source_file'] = pdf.name
                doc.metadata['file_type'] = 'pdf'

            all_docs.extend(document)

        except Exception as e:
            print(f" Error:{e}")
        
    print(f"total {len(all_docs)} loaded")
    return all_docs



 
 

In [94]:
documents = process_all_pdfs('../data/pdf')

Found 6 pdf to process
processing
processing
processing
processing
processing
processing
total 21 loaded


In [95]:
### TExt splitting

def split_docs_fn(documents,chunk_size =1000,chunk_overlap = 200):
    """splits document into smaller chunks for better rag performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = 200,
        length_function = len,
        separators=["\n\n","\n",""," "]
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"split {len(documents)} into {len(split_docs)} chunks")

    if split_docs:
        print(f"content : {split_docs[0].page_content[:1000]}")
        print(f"Metadata {split_docs[0].metadata}")
    
    return split_docs

In [96]:
chunks = split_docs_fn(documents)
chunks

split 21 into 42 chunks
content : CamScanner
Metadata {'producer': 'intsig.com pdf producer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'assignment', 'author': 'CamScanner', 'subject': 'assignment', 'moddate': '', 'keywords': '', 'source': '..\\data\\pdf\\assignment .pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'assignment .pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'intsig.com pdf producer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'assignment', 'author': 'CamScanner', 'subject': 'assignment', 'moddate': '', 'keywords': '', 'source': '..\\data\\pdf\\assignment .pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'assignment .pdf', 'file_type': 'pdf'}, page_content='CamScanner'),
 Document(metadata={'producer': 'intsig.com pdf producer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'assignment', 'author': 'CamScanner', 'subject': 'assignment', 'moddate': '', 'keywords': '', 'source': '..\\data\\pdf\\assignment .pdf', 'total_pages': 2, 'page': 1, 'page_label': '2', 'source_file': 'assignment .pdf', 'file_type': 'pdf'}, page_content='CamScanner'),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-11-09T13:00:10+05:45', 'author': '', 'moddate': '2025-11-09T13:00:10+05:45', 'title': 'Microsoft Word - CT304 COMPUTER NETWORKS', 'source': '..\\

### Embedding and VectorDb

In [97]:
import numpy as np 
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [98]:
class EmbeddingManager:
    """Handles documents embedding generation using SentenceTransformer"""
    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name:HuggingFace model name for sentence embeddings
        """

        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the sentenceTransformer model"""
        try:
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded with Embedding dimension : {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name} : {e}")
            raise

    def generate_embeddings(self,texts:List[str]) -> np.ndarray:
        """Generate embeddings for a list of texts

            Args:
            texta:List of text strings to embed

            Returns:
            numpy array of embeddings with shape (len(textss),embedding_dim)
        """

        if not self.model:
            raise ValueError("Model not found")
        
        print(f"Generating embeddings")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
    # def get_embedding_dimension(self) ->int:
    #     """Get the embedding dimension of loaded model"""
    #     if not self.model:
    #         raise ValueError("Mmodel not loaded.")
        
    #     return self.model.get_sentence_embedding_dimension()


embedding_manager = EmbeddingManager()

embedding_manager


Model loaded with Embedding dimension : 384


### Vector Store

In [99]:
class VectorStore:
    """Manages document embeddings in a chromaDB vector store"""

    def __init__(self,collection_name:str = "pdf_documents",persist_directory:str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name:name of chromaDB collection
            persist_directory : Directory to persist the vector store
        """
        
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""

        try:
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata={"description":"PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized Collection:{self.collection_name}")
            print(f"Existing documents in collection:{self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
        documents:List of Langchain documents
        embeddings : corresponding embeddings for documents
        """

        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match no of Embeddings")
        
        print(f"Adding {len(documents)} documents to vector store ...")
        #prepare data for chromaDB
        ids =[]
        metadatas = []
        document_texts = []
        embeddings_list = []

        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            document_texts.append(doc.page_content)

            embeddings_list.append(embedding.tolist())
        self.collection.add(
            ids=ids,
            embeddings=embeddings_list,
            documents=document_texts,
            metadatas=metadatas
        )
        print(document_texts,embeddings_list)
        print(self.collection.count())

        

vector_store = VectorStore()
vector_store

Vector store initialized Collection:pdf_documents
Existing documents in collection:42


In [100]:
## text to embeddings
texts = [doc.page_content for doc in chunks]
texts

['CamScanner',
 'CamScanner',
 'COMPUTER NETWORKS \nENCT 304 \n    \n \nCourse Objectives: \nThe objective of this course is to provide fundamental concepts and principles of \ncomputer networks. It focuses on various network architectures, protocols, standards, \nand the networking device functionalities. The course also aims to equip students with \nthe skills to design, implement, test and troubleshoot basic network configurations. \n \n1 Introduction         (5 hours)   \n1.1 Computer network, importance and applications \n1.2 Types of computer networks \n1.3 Network topologies \n1.4 Client/server and P2P networks \n1.5 Protocols and standards, need of standardization \n1.6 Layered network architecture, OSI and TCP/IP reference models \n1.7 Addressing at different layers \n1.8 Networking devices: Repeater, hub, bridge, switch, router, gateway \n \n2 Physical Layer      (5 hours)  \n2.1 Network monitoring: Bandwidth, throughput, delay, round-trip-time \n2.2 Multiplexing - FDM, TDM, 

In [101]:
embeddings = embedding_manager.generate_embeddings(texts)

#store in vector db
vector_store.add_documents(chunks,embeddings)


Generating embeddings


Batches: 100%|██████████| 2/2 [00:01<00:00,  1.95it/s]


Generated embeddings with shape: (42, 384)
Adding 42 documents to vector store ...
['CamScanner', 'CamScanner', 'COMPUTER NETWORKS \nENCT 304 \n    \n \nCourse Objectives: \nThe objective of this course is to provide fundamental concepts and principles of \ncomputer networks. It focuses on various network architectures, protocols, standards, \nand the networking device functionalities. The course also aims to equip students with \nthe skills to design, implement, test and troubleshoot basic network configurations. \n \n1 Introduction         (5 hours)   \n1.1 Computer network, importance and applications \n1.2 Types of computer networks \n1.3 Network topologies \n1.4 Client/server and P2P networks \n1.5 Protocols and standards, need of standardization \n1.6 Layered network architecture, OSI and TCP/IP reference models \n1.7 Addressing at different layers \n1.8 Networking devices: Repeater, hub, bridge, switch, router, gateway \n \n2 Physical Layer      (5 hours)  \n2.1 Network monitori

In [102]:
vector_store.collection.count()

84

## RAG Retriver Pipeline

In [112]:
class RagRetriver:
    """Handles query based retrieval from the vector store"""

    def __init__(self,vector_store:VectorStore,embedding_manager:EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store:Vector store containing document embeddings
            embedding_manager : Manager for generating query embeddings
        """

        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    
    def retrieve(self,query:str,top_k:int = 5, score_threshold:float = 0.0) -> List[Dict[str,Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query:The Search query
            top_k : Number of top results to return
            score_threshold:Minimum similarity score threshold
        
        Returns:
            List of dictionaries containing retrieved documents and metadata

        """

        print(f"Retrieving documents for query : {query}")
        print(f"Top K : {top_k},score threshold {score_threshold}")

        #generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        #search in vector DB

        if self.vector_store.collection is None:
            raise RuntimeError("Vector store collection is not initialized")
        # print(self.vector_store.collection.count(),"sa")
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            print(results)

            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                print("Hello world")
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id,document,metadata,distance) in enumerate (zip(ids,documents,metadatas,distances)):
                    #convert distances into similarity score

                    similarity_score = 1 / (1 + distance)

                    print(similarity_score)
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,
                            'content':document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance':distance,
                            'rank':i+1

                        })

                print(f"Retrieved {len(retrieved_docs)} documents")
            else:
                print('no document found')

            return retrieved_docs
                
        
        except Exception as e:
            print(f"Error retrieving : {e}")
            return []



In [114]:
rag_retriever = RagRetriver(vector_store,embedding_manager)
rag_retriever.retrieve('what is marks distribution for computer network?')

Retrieving documents for query : what is marks distribution for computer network?
Top K : 5,score threshold 0.0
Generating embeddings


Batches: 100%|██████████| 1/1 [00:00<00:00, 140.16it/s]

Generated embeddings with shape: (1, 384)
{'ids': [['doc_845ad955_10', 'doc_a04330a8_10', 'doc_829c965f_40', 'doc_2984d68b_40', 'doc_0ccd21bc_18']], 'embeddings': None, 'documents': [['Chapter Hours Marks distribution* \n1 5 6 \n2 5 6 \n3 8 12 \n4 12 15 \n5 5 6 \n6 6 9 \n7 4 6 \nTotal 45 60 \n* There may be minor deviations in marks distribution. \n \nReferences  \n1. Forouzan, B.\u202fA. (2012). Data communications and networking. McGraw‑Hill.  \n2. Kurose, J.\u202fF., Ross, K.\u202fW. (2017). Computer networking: A top‑down \napproach. Pearson.  \n3. Tanenbaum, A.S., Feamster, N. (2021). Computer networks. Pearson. \n4. Stevens, W.R., Fall, K.R. (2011). TCP/IP illustrated: The protocols. \nAddison‑Wesley.  \n5. RFCs and online resources for protocol specifications.', 'Chapter Hours Marks distribution* \n1 5 6 \n2 5 6 \n3 8 12 \n4 12 15 \n5 5 6 \n6 6 9 \n7 4 6 \nTotal 45 60 \n* There may be minor deviations in marks distribution. \n \nReferences  \n1. Forouzan, B.\u202fA. (2012). Data

[{'id': 'doc_845ad955_10',
  'content': 'Chapter Hours Marks distribution* \n1 5 6 \n2 5 6 \n3 8 12 \n4 12 15 \n5 5 6 \n6 6 9 \n7 4 6 \nTotal 45 60 \n* There may be minor deviations in marks distribution. \n \nReferences  \n1. Forouzan, B.\u202fA. (2012). Data communications and networking. McGraw‑Hill.  \n2. Kurose, J.\u202fF., Ross, K.\u202fW. (2017). Computer networking: A top‑down \napproach. Pearson.  \n3. Tanenbaum, A.S., Feamster, N. (2021). Computer networks. Pearson. \n4. Stevens, W.R., Fall, K.R. (2011). TCP/IP illustrated: The protocols. \nAddison‑Wesley.  \n5. RFCs and online resources for protocol specifications.',
  'metadata': {'producer': 'Microsoft: Print To PDF',
   'author': '',
   'creator': 'PyPDF',
   'file_type': 'pdf',
   'source_file': 'CN.pdf',
   'content_length': 559,
   'moddate': '2025-11-09T13:00:10+05:45',
   'title': 'Microsoft Word - CT304 COMPUTER NETWORKS',
   'creationdate': '2025-11-09T13:00:10+05:45',
   'total_pages': 4,
   'doc_index': 10,
   's